In [73]:
import numpy as np
import pandas as pd

In [74]:
match = pd.read_csv('ODI_Match_Info.csv')
delivery = pd.read_csv('ODI_Match_Data.csv', low_memory=False)

In [75]:
match.info

<bound method DataFrame.info of            id   season          city        date        team1        team2  \
0     1389389  2023/24        Indore  2023/09/24        India    Australia   
1     1336129     2023    Nottingham  2023/09/23      England      Ireland   
2     1395701     2023         Dhaka  2023/09/23  New Zealand   Bangladesh   
3     1389388  2023/24    Chandigarh  2023/09/22    Australia        India   
4     1395700     2023         Dhaka  2023/09/21  New Zealand   Bangladesh   
...       ...      ...           ...         ...          ...          ...   
2374    65634  2002/03        Sydney  2003/01/09    Sri Lanka    Australia   
2375    64817  2002/03    Wellington  2003/01/08  New Zealand        India   
2376    64816  2002/03    Queenstown  2003/01/04        India  New Zealand   
2377    64815  2002/03  Christchurch  2003/01/01        India  New Zealand   
2378    64814  2002/03        Napier  2002/12/29  New Zealand        India   

      toss_winner toss_decision

In [76]:
delivery.info

<bound method DataFrame.info of          match_id   season  start_date                           venue  \
0         1389389  2023/24  2023-09-24  Holkar Cricket Stadium, Indore   
1         1389389  2023/24  2023-09-24  Holkar Cricket Stadium, Indore   
2         1389389  2023/24  2023-09-24  Holkar Cricket Stadium, Indore   
3         1389389  2023/24  2023-09-24  Holkar Cricket Stadium, Indore   
4         1389389  2023/24  2023-09-24  Holkar Cricket Stadium, Indore   
...           ...      ...         ...                             ...   
1265098     64814  2002/03  2002-12-29             McLean Park, Napier   
1265099     64814  2002/03  2002-12-29             McLean Park, Napier   
1265100     64814  2002/03  2002-12-29             McLean Park, Napier   
1265101     64814  2002/03  2002-12-29             McLean Park, Napier   
1265102     64814  2002/03  2002-12-29             McLean Park, Napier   

         innings  ball batting_team bowling_team     striker   non_striker  \
0

In [77]:
match.shape

(2379, 18)

In [78]:
delivery.shape

(1265103, 23)

In [79]:
print(delivery.columns)

Index(['match_id', 'season', 'start_date', 'venue', 'innings', 'ball',
       'batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler',
       'runs_off_bat', 'extras', 'wides', 'noballs', 'byes', 'legbyes',
       'penalty', 'wicket_type', 'player_dismissed', 'other_wicket_type',
       'other_player_dismissed', 'cricsheet_id'],
      dtype='object')


In [80]:
delivery['total_runs'] = delivery['runs_off_bat'] + delivery['extras'].fillna(0)


In [81]:
finalscoredf = delivery.groupby(['match_id', 'innings']).sum()['total_runs'].reset_index()


In [82]:
finalscoredf

,match_id,innings,total_runs
0,64814,1,254
1,64814,2,219
2,64815,1,108
3,64815,2,109
4,64816,1,122
...,...,...,...
4696,1390346,1,268
4697,1390346,2,209
4698,1395700,1,136
4699,1395701,1,254


In [83]:
finalscoredf =finalscoredf[finalscoredf['innings']==1]

In [84]:
matchdf = match.merge(finalscoredf[['match_id', 'total_runs']], left_on='id', right_on='match_id')

In [85]:
teams = [
    'Australia',
    'India',
    'Bangladesh',
    'New Zealand',
    'South Africa',
    'England',
    'West Indies',
    'Afghanistan',
    'Pakistan',
    'Sri Lanka'    
]

In [86]:
matchdf = matchdf[matchdf['team1'].isin(teams)]
matchdf = matchdf[matchdf['team2'].isin(teams)]

In [87]:
matchdf = matchdf[matchdf['dl_applied']==0]

In [88]:
matchdf = matchdf[['match_id','city','winner','total_runs']]

In [89]:
deliverydf = matchdf.merge(delivery,on='match_id')

In [90]:
deliverydf = deliverydf[deliverydf['innings'] == 2]

In [91]:
pd.set_option('display.max_columns', None)
deliverydf=deliverydf.dropna(subset=['city','winner'])

In [92]:
print(deliverydf['total_runs_y'].dtype)
deliverydf['currentscore'] = deliverydf.groupby('match_id')['total_runs_y'].cumsum()

int64


In [93]:
deliverydf['runs_left'] = deliverydf['total_runs_x'] - deliverydf['currentscore'] + 1
deliverydf

,match_id,city,winner,total_runs_x,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,cricsheet_id,total_runs_y,currentscore,runs_left
303,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.1,Bangladesh,New Zealand,Tamim Iqbal,Liton Das,TA Boult,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1395701,0,0,255
304,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.2,Bangladesh,New Zealand,Tamim Iqbal,Liton Das,TA Boult,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1395701,0,0,255
305,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.3,Bangladesh,New Zealand,Tamim Iqbal,Liton Das,TA Boult,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1395701,0,0,255
306,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.4,Bangladesh,New Zealand,Tamim Iqbal,Liton Das,TA Boult,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1395701,1,1,254
307,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.5,Bangladesh,New Zealand,Liton Das,Tamim Iqbal,TA Boult,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1395701,0,1,254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789641,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,42.6,India,New Zealand,J Srinath,Z Khan,PA Hitchcock,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64814,0,216,39
789642,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,43.1,India,New Zealand,Z Khan,J Srinath,KD Mills,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64814,1,217,38
789643,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,43.2,India,New Zealand,J Srinath,Z Khan,KD Mills,0,0,NaN,NaN,NaN,NaN,NaN,caught,J Srinath,NaN,NaN,64814,0,217,38
789644,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,43.3,India,New Zealand,Z Khan,A Nehra,KD Mills,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64814,2,219,36


In [94]:
deliverydf['over'] = deliverydf['ball'].apply(lambda x:str(x).split(".")[0])
deliverydf['ballofover'] = deliverydf['ball'].apply(lambda x:str(x).split(".")[1])
deliverydf



,match_id,city,winner,total_runs_x,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,cricsheet_id,total_runs_y,currentscore,runs_left,over,ballofover
303,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.1,Bangladesh,New Zealand,Tamim Iqbal,Liton Das,TA Boult,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1395701,0,0,255,0,1
304,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.2,Bangladesh,New Zealand,Tamim Iqbal,Liton Das,TA Boult,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1395701,0,0,255,0,2
305,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.3,Bangladesh,New Zealand,Tamim Iqbal,Liton Das,TA Boult,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1395701,0,0,255,0,3
306,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.4,Bangladesh,New Zealand,Tamim Iqbal,Liton Das,TA Boult,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1395701,1,1,254,0,4
307,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.5,Bangladesh,New Zealand,Liton Das,Tamim Iqbal,TA Boult,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1395701,0,1,254,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789641,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,42.6,India,New Zealand,J Srinath,Z Khan,PA Hitchcock,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64814,0,216,39,42,6
789642,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,43.1,India,New Zealand,Z Khan,J Srinath,KD Mills,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64814,1,217,38,43,1
789643,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,43.2,India,New Zealand,J Srinath,Z Khan,KD Mills,0,0,NaN,NaN,NaN,NaN,NaN,caught,J Srinath,NaN,NaN,64814,0,217,38,43,2
789644,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,43.3,India,New Zealand,Z Khan,A Nehra,KD Mills,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64814,2,219,36,43,3


In [95]:
deliverydf['over'] = pd.to_numeric(deliverydf['over'], errors='coerce')
deliverydf['ballofover'] = pd.to_numeric(deliverydf['ballofover'], errors='coerce')
deliverydf['balls_left'] = 300 - (deliverydf['over'] * 6 + deliverydf['ballofover'])

deliverydf

,match_id,city,winner,total_runs_x,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,cricsheet_id,total_runs_y,currentscore,runs_left,over,ballofover,balls_left
303,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.1,Bangladesh,New Zealand,Tamim Iqbal,Liton Das,TA Boult,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1395701,0,0,255,0,1,299
304,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.2,Bangladesh,New Zealand,Tamim Iqbal,Liton Das,TA Boult,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1395701,0,0,255,0,2,298
305,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.3,Bangladesh,New Zealand,Tamim Iqbal,Liton Das,TA Boult,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1395701,0,0,255,0,3,297
306,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.4,Bangladesh,New Zealand,Tamim Iqbal,Liton Das,TA Boult,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1395701,1,1,254,0,4,296
307,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.5,Bangladesh,New Zealand,Liton Das,Tamim Iqbal,TA Boult,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1395701,0,1,254,0,5,295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789641,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,42.6,India,New Zealand,J Srinath,Z Khan,PA Hitchcock,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64814,0,216,39,42,6,42
789642,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,43.1,India,New Zealand,Z Khan,J Srinath,KD Mills,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64814,1,217,38,43,1,41
789643,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,43.2,India,New Zealand,J Srinath,Z Khan,KD Mills,0,0,NaN,NaN,NaN,NaN,NaN,caught,J Srinath,NaN,NaN,64814,0,217,38,43,2,40
789644,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,43.3,India,New Zealand,Z Khan,A Nehra,KD Mills,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64814,2,219,36,43,3,39


In [96]:
deliverydf['player_dismissed'] = deliverydf['player_dismissed'].fillna("0")

In [97]:
deliverydf['player_dismissed'] = deliverydf['player_dismissed'].apply(lambda x:x if x == "0" else "1")

In [98]:
deliverydf['player_dismissed'] = deliverydf['player_dismissed'].astype('int')
deliverydf

,match_id,city,winner,total_runs_x,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,cricsheet_id,total_runs_y,currentscore,runs_left,over,ballofover,balls_left
303,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.1,Bangladesh,New Zealand,Tamim Iqbal,Liton Das,TA Boult,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,1395701,0,0,255,0,1,299
304,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.2,Bangladesh,New Zealand,Tamim Iqbal,Liton Das,TA Boult,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,1395701,0,0,255,0,2,298
305,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.3,Bangladesh,New Zealand,Tamim Iqbal,Liton Das,TA Boult,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,1395701,0,0,255,0,3,297
306,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.4,Bangladesh,New Zealand,Tamim Iqbal,Liton Das,TA Boult,1,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,1395701,1,1,254,0,4,296
307,1395701,Dhaka,New Zealand,254,2023,2023-09-23,"Shere Bangla National Stadium, Mirpur",2,0.5,Bangladesh,New Zealand,Liton Das,Tamim Iqbal,TA Boult,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,1395701,0,1,254,0,5,295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789641,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,42.6,India,New Zealand,J Srinath,Z Khan,PA Hitchcock,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,64814,0,216,39,42,6,42
789642,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,43.1,India,New Zealand,Z Khan,J Srinath,KD Mills,1,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,64814,1,217,38,43,1,41
789643,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,43.2,India,New Zealand,J Srinath,Z Khan,KD Mills,0,0,NaN,NaN,NaN,NaN,NaN,caught,1,NaN,NaN,64814,0,217,38,43,2,40
789644,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,43.3,India,New Zealand,Z Khan,A Nehra,KD Mills,2,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,64814,2,219,36,43,3,39


In [99]:
wickets = deliverydf.groupby('match_id')['player_dismissed'].cumsum()
deliverydf['wickets_left'] = 10 - wickets
deliverydf.tail(6)

,match_id,city,winner,total_runs_x,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,cricsheet_id,total_runs_y,currentscore,runs_left,over,ballofover,balls_left,wickets_left
789640,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,42.5,India,New Zealand,J Srinath,Z Khan,PA Hitchcock,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,64814,0,216,39,42,5,43,2
789641,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,42.6,India,New Zealand,J Srinath,Z Khan,PA Hitchcock,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,64814,0,216,39,42,6,42,2
789642,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,43.1,India,New Zealand,Z Khan,J Srinath,KD Mills,1,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,64814,1,217,38,43,1,41,2
789643,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,43.2,India,New Zealand,J Srinath,Z Khan,KD Mills,0,0,NaN,NaN,NaN,NaN,NaN,caught,1,NaN,NaN,64814,0,217,38,43,2,40,1
789644,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,43.3,India,New Zealand,Z Khan,A Nehra,KD Mills,2,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,64814,2,219,36,43,3,39,1
789645,64814,Napier,New Zealand,254,2002/03,2002-12-29,"McLean Park, Napier",2,43.4,India,New Zealand,Z Khan,A Nehra,KD Mills,0,0,NaN,NaN,NaN,NaN,NaN,caught,1,NaN,NaN,64814,0,219,36,43,4,38,0


In [100]:
deliverydf['crr'] = (deliverydf['currentscore']*6)/(300 - deliverydf['balls_left'])

In [101]:
deliverydf['rrr'] = (deliverydf['runs_left']*6)/deliverydf['balls_left']

In [102]:
def result(row):
    return 1 if row['batting_team'] == row['winner'] else 0

deliverydf['result'] = deliverydf.apply(result,axis=1)

In [103]:
finaldf = deliverydf[['batting_team','bowling_team','city','runs_left','balls_left','wickets_left','total_runs_x','crr','rrr','result']]

In [104]:
finaldf = finaldf.sample(finaldf.shape[0])

finaldf

,batting_team,bowling_team,city,runs_left,balls_left,wickets_left,total_runs_x,crr,rrr,result
351606,England,Australia,Southampton,179,135,5,298,4.363636,7.955556,0
232549,England,Pakistan,Leeds,98,130,6,247,5.294118,4.523077,1
26449,England,South Africa,Bloemfontein,105,129,7,298,6.807018,4.883721,0
680848,West Indies,New Zealand,Christchurch,195,229,8,276,6.929577,5.109170,0
89898,Sri Lanka,Pakistan,Karachi,298,289,10,305,4.363636,6.186851,0
...,...,...,...,...,...,...,...,...,...,...
745233,Sri Lanka,Australia,Colombo,165,213,9,233,4.758621,4.647887,0
156642,India,England,London,274,251,10,322,6.000000,6.549801,0
616549,Bangladesh,India,Mirpur,260,261,9,284,3.846154,5.977011,0
230765,Afghanistan,Bangladesh,Mirpur,180,183,8,265,4.410256,5.901639,0


In [105]:
finaldf.dropna(inplace=True)
finaldf=finaldf[finaldf['balls_left']!=0]
finaldf.sample()

,batting_team,bowling_team,city,runs_left,balls_left,wickets_left,total_runs_x,crr,rrr,result
504732,Australia,Pakistan,Brisbane,238,250,9,274,4.44,5.712,1


In [106]:
finaldf.columns

Index(['batting_team', 'bowling_team', 'city', 'runs_left', 'balls_left',
       'wickets_left', 'total_runs_x', 'crr', 'rrr', 'result'],
      dtype='object')

In [107]:
finaldf.to_csv('Win_Predictor_Data.csv')

In [108]:
X = finaldf.drop(['runs_left', 'balls_left'], axis=1)
y = finaldf.iloc[:, -1]
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [109]:
X_train.describe

<bound method NDFrame.describe of         batting_team  bowling_team               city  wickets_left  \
420700   West Indies         India            Chennai             9   
624932       England     Sri Lanka            Antigua            10   
409305  South Africa   New Zealand             Napier             8   
567448      Pakistan    Bangladesh            Karachi            10   
190782         India    Bangladesh         Birmingham             9   
...              ...           ...                ...           ...   
498927   West Indies     Australia           Brisbane            10   
477403      Pakistan       England  Chester-le-Street             7   
91257      Sri Lanka    Bangladesh            Colombo             7   
649300         India  South Africa     Port Elizabeth             4   
66838        England         India               Pune             2   

        total_runs_x       crr       rrr  result  
420700           267  5.142857  5.376344       0  
624932     

In [110]:
y_train.describe

<bound method NDFrame.describe of 420700    0
624932    0
409305    1
567448    1
190782    1
         ..
498927    0
477403    0
91257     1
649300    0
66838     0
Name: result, Length: 249945, dtype: int64>

In [111]:

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier


trf = ColumnTransformer([
    ('trf',OneHotEncoder(sparse_output=False,drop='first'),['batting_team','bowling_team','city'])
]
,remainder='passthrough')

In [112]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

In [113]:

# Different models for classification
models = {
   #'LogisticRegression': LogisticRegression(solver='newton-cg', random_state=1),
    'RandomForestClassifier': RandomForestClassifier(n_estimators=100, random_state=1),
    #'XGBClassifier': XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=1)
}

# Dictionary to store results
results = {}

# Iterate through each model and fit the data
for name, model in models.items():
    # Create a pipeline
    pipe = Pipeline(steps=[
        ('step1', trf),
        ('step2', model)
    ])

In [114]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('step1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('trf',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['batting_team',
                                                   'bowling_team', 'city'])])),
                ('step2',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=Non...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=100, n_jobs=None,
                               num_parallel_tree=None, random_state=1, ...))])

In [115]:
y_pred = pipe.predict(X_test)

In [116]:

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
    
    # Store the result
results[name] = {
        'Accuracy': accuracy
}

results

{'XGBClassifier': {'Accuracy': 1.0}}

In [117]:
X.columns

Index(['batting_team', 'bowling_team', 'city', 'wickets_left', 'total_runs_x',
       'crr', 'rrr', 'result'],
      dtype='object')

In [118]:
X['city'].unique()

array(['Southampton', 'Leeds', 'Bloemfontein', 'Christchurch', 'Karachi',
       'Antigua', 'Auckland', 'Manchester', 'Abu Dhabi', 'Mirpur',
       'St Vincent', 'Centurion', 'Hyderabad', 'Paarl', 'Visakhapatnam',
       'Guwahati', 'Chester-le-Street', 'Cuttack', 'Ahmedabad',
       'St Lucia', 'Chandigarh', 'Mumbai', 'Brisbane', 'Napier', 'Cairns',
       'London', 'Durban', 'Perth', 'Birmingham', 'Colombo',
       'Potchefstroom', 'Dunedin', 'Hobart', 'Delhi', 'Port Elizabeth',
       'Cardiff', 'Sharjah', 'St Kitts', 'Wellington', 'Lahore',
       'Bridgetown', 'Hamilton', 'Johannesburg', 'Fatullah', 'Taunton',
       'Nottingham', 'Dominica', 'Guyana', 'Gwalior', 'Kanpur', 'Nagpur',
       'Trinidad', 'Mount Maunganui', 'Pune', 'Jamaica', 'Chennai',
       'Ranchi', 'Dublin', 'Kochi', 'Melbourne', 'Lucknow', 'Sydney',
       'Adelaide', 'Chattogram', 'Barbados', 'Multan', 'Cape Town',
       'Canberra', 'Kandy', 'Jaipur', 'Queenstown', 'Dharmasala',
       'Indore', 'Kolkata', 'Gr